In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import drive
drive.mount('/content/drive')

import torch
from transformers import pipeline
# Define the directory where the model is saved
load_directory = "/content/drive/Shareddrives/517 nlp project/Models/Llama-3.2-1B-Instruct"

# # Load the model and tokenizer from the specified directory
# tokenizer = AutoTokenizer.from_pretrained(load_directory)
# model = AutoModelForCausalLM.from_pretrained(load_directory).to("cuda")

# # Ensure the tokenizer has a pad_token
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
#     model.resize_token_embeddings(len(tokenizer))  # Resize model's embedding matrix if a new token is added

# print("Model and tokenizer successfully loaded from Google Drive!")

# def get_response(prompt, max_length=1024):
#     inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")
#     output = model.generate(
#         inputs.input_ids,
#         max_length=max_length,
#         temperature=0.1,  # Lower temperature for more focused answers
#         do_sample=True,
#         num_beams=3,      # Add beam search for better coherence
#         top_p=0.9,        # Nucleus sampling
#         repetition_penalty=1.2  # Increased penalty for repetition
#     )
#     response = tokenizer.decode(output[0], skip_special_tokens=True)
#     return response

# print(get_response("Hello, how are you?"))

pipe = pipeline(
    "text-generation",
    model=load_directory,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

def get_response(prompt, max_length=512):
    messages = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt}
    ]
    # add chat template
    # messages = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
    outputs = pipe(
        messages,
        max_new_tokens=max_length,
        temperature=0.7,
        repetition_penalty=1.1,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    # print(outputs)
    return outputs[0]["generated_text"][-1]['content']

# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
# outputs = pipe(
#     messages,
#     max_new_tokens=256,
# )
# print(outputs[0]["generated_text"][-1])

print(get_response("how are you?"))

In [ ]:
import random


def extract_answer(response):
    """Extract the numerical answer from between [ANSWER] tags"""
    try:
        # Look for answer between tags
        import re
        pattern = r'\[ANSWER\]<(\d+\.?\d*)>\[\/ANSWER\]'
        match = re.search(pattern, response)
        if match:
            return float(match.group(1))

        # Fallback: look for last number after "answer is" or similar phrases
        patterns = [
            r'ANSWER<(\d+\.?\d*)>\[\/ANSWER\]'
            # r'answer is[^\d]*(\d+\.?\d*)',
            # r'answer:[^\d]*(\d+\.?\d*)',
            # r'equals[^\d]*(\d+\.?\d*)',
            # r'=\s*(\d+\.?\d*)'
        ]

        for pattern in patterns:
            match = re.search(pattern, response.lower())
            if match:
                return float(match.group(1))

        # Final fallback: last number in response
        numbers = re.findall(r'\d+\.?\d*', response)
        if numbers:
            return float(numbers[-1])

        return None
    except:
        return None

def create_demo_text(cot_flag=True):
    x, z, y = [], [], []
    # example sentences ...
    if 1:

        x.append("There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?")
        z.append("Step 1, There are 15 trees originally, Then there were 21 trees after some more were planted. Step 2, So there must have been 21 - 15 = 6.")
        y.append("6")

        x.append("If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?")
        z.append("Step 1, There are originally 3 cars, and  2 more cars arrive. Step 2, 3 + 2 = 5.")
        y.append("5")

        x.append("Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?")
        z.append("Step 1, Originally, Leah had 32 chocolates, and her sister had 42. Step 2, So in total they had 32 + 42 = 74. Step 3, After eating 35, they had 74 - 35 = 39.")
        y.append("39")

        # x.append("Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?")
        # z.append("Step 1, Jason started with 20 lollipops. Then he had 12 after giving some to Denny. Step 2, So he gave Denny 20 - 12 = 8.")
        # y.append("8")


    else:
        raise ValueError("dataset is not properly defined ...")

    # randomize order of the examples ...
    index_list = list(range(len(x)))
    random.shuffle(index_list)

    # Concatenate demonstration examples ...
    direct_answer_trigger_for_fewshot = "The answer is "
    demo_text = ""
    for i in index_list:
        demo_text += "Q: " + x[i] + "\nA: " + z[i] + " " + \
                  direct_answer_trigger_for_fewshot + "[ANSWER]<" + y[i] + ">[/ANSWER].\n\n"
    return demo_text

def generate_math_prompt(question):
    prompt = f"""Solve a math word problem step by step. Finally, state the answer in the format: [ANSWER]<number>[/ANSWER]
Here are some examples:
{create_demo_text()}
Now solve the following math word problem:
Q: {question}
A: """
    return prompt

def generate_evaluate_prompt(question, y):
    prompt = f"""Evaluate if the given reasoning can lead to the right solution to the question. Choose one word from (correct, likely, impossible) to indicate your evaluation of the reasoning's quality. Do not output anything else.
Question: {question}
Reasoning: {y}
Evaluation: """
    return prompt

def generate_evaluate_last_state_prompt(question, y):
    prompt = f"""Evaluate if the given reasoning is the right solution to the question with the right format of the answer in [ANSWER]<number>[/ANSWER]. Choose one word from (correct, likely, impossible) to indicate your evaluation of the reasoning's quality. Answer 'impossible' if there's no answer in the reasoning. Do not output anything else.
Question: {question}
Reasoning: {y}
Evaluation: """
    return prompt

print(create_demo_text())
prompt = generate_math_prompt("julia played tag with 16 kids on monday. if she played tag with 12 more kids on monday than on tuesday how many kids did she play with on tuesday?")
print(prompt)
print(get_response(prompt))
print(generate_evaluate_prompt("a", "b"))



7.0


In [ ]:
import numpy as np
def get_new_ys(x, ys, step, n_generate_sample):
    '''
    x is the question prompt
    ys is the current output candidates (from step 1 to step {step-1})
    step is the current step
    n_generate_sample is the number of new output candidates to generate
    return the new output candidates (from step 1 to step {step})
    '''
    new_ys = []
    for y in ys:
        for _ in range(n_generate_sample):
            if "[ANSWER]" in y:
                new_ys.append(y)
                continue
            prompt = x + y + "Step " + str(step) + ", "
            # print(prompt)
            new_y = get_response(prompt).replace("\n", "")
            # clean new_y and only get the thought of this step
            ## cut if there's Step {step+1} or The answer is
            new_y = new_y.split("Step " + str(step+1))[0]
            # new_y = new_y.split("The answer is ")[0]
            new_ys.append(y + "Step " + str(step) + ", " + new_y)
    return new_ys

def get_new_ys_last_step(x, ys, n_generate_sample):
    '''
    x is the question prompt
    ys is the current output candidates (from step 1 to step {step-1})
    step is the current step
    n_generate_sample is the number of new output candidates to generate
    return the new output candidates (from step 1 to step {step})
    '''
    new_ys = []
    for y in ys:
        for _ in range(n_generate_sample):
            prompt = x + y + "The answer is [ANSWER]<"
            # print(prompt)
            new_y = get_response(prompt)
            # clean new_y and only get the thought of this step
            ## cut if there's Step {step+1} or The answer is

            new_ys.append(y + "The answer is [ANSWER]<" + new_y)
    return new_ys

def evaluate_state(question, y):
    prompt = generate_evaluate_prompt(question, y)
    response = get_response(prompt, max_length=128).lower()
    # print(response)

    if 'impossible' in response or 'incorrect' in response or 'unlikely' in response:
        return 0
    elif 'likely' in response:
        return 0.5
    elif 'correct' in response:
        return 1
    else:
        return 0.01

def evaluate_state_last_step(question, y):
    prompt = generate_evaluate_last_state_prompt(question, y)
    response = get_response(prompt, max_length=128).lower()
    # print(response)
    if 'impossible' in response or 'incorrect' in response or 'unlikely' in response:
        return 0
    elif 'likely' in response:
        return 0.5
    elif 'correct' in response:
        return 1
    else:
        return 0.01

def solve(question, n_generate_sample=10, n_select_sample=5, depth=3, verbose=False):
    global pipe
    x = generate_math_prompt(question)
    ys = ['']  # current output candidates
    infos = []
    for step in range(1, depth+1):
        if step < depth:
            # generation
            new_ys = get_new_ys(x, ys, step, n_generate_sample)
            if verbose:
                print(new_ys)
            ids = list(range(len(new_ys)))

            # evaluation
            values = [evaluate_state(question, new_y) for new_y in new_ys]

            # selection
            ps = np.array(values) / sum(values)
            select_ids = np.random.choice(ids, size=n_select_sample, p=ps).tolist()
            select_new_ys = [new_ys[select_id] for select_id in select_ids]

            # log
            sorted_new_ys, sorted_values = zip(*sorted(zip(new_ys, values), key=lambda x: x[1], reverse=True))
            if verbose:
                print(f'-- new_ys --: {sorted_new_ys}\n-- sol values --: {sorted_values}\n-- choices --: {select_new_ys}\n')

            infos.append({'step': step, 'x': x, 'ys': ys, 'new_ys': new_ys, 'values': values, 'select_new_ys': select_new_ys})
            ys = select_new_ys
        else:
            # generation
            new_ys = get_new_ys_last_step(x, ys, n_generate_sample)
            if verbose:
                print(new_ys)
            ids = list(range(len(new_ys)))

            # evaluation
            values = [evaluate_state_last_step(question, new_y) for new_y in new_ys]
            if verbose:
                print(values)

            # selection
            ps = np.array(values) / sum(values)
            select_ids = np.random.choice(ids, size=1, p=ps).tolist()
            select_new_ys = [new_ys[select_id] for select_id in select_ids]

            # log
            sorted_new_ys, sorted_values = zip(*sorted(zip(new_ys, values), key=lambda x: x[1], reverse=True))
            if verbose:
                print(f'-- new_ys --: {sorted_new_ys}\n-- sol values --: {sorted_values}\n-- choices --: {select_new_ys}\n')

            infos.append({'step': step, 'x': x, 'ys': ys, 'new_ys': new_ys, 'values': values, 'select_new_ys': select_new_ys})
            ys = select_new_ys

    if verbose:
        print(ys)
    return ys, {'steps': infos}


In [ ]:
ys, info = solve("julia played tag with 18 kids on monday. she played tag with 10 kids on tuesday. how many more kids did she play with on monday than on tuesday?",
      6,
      3,
      3)
print(ys[0])
answer = extract_answer(ys[0])
print(answer)
# print(info)

['Step 1, To find out how many more kids Julia played with on Monday than on Tuesday, we need to subtract the number of kids she played with on Tuesday from the number of kids she played with on Monday.Step 1: Identify the numbers.Number of kids Julia played with on Monday: 18Number of kids Julia played with on Tuesday: 10', 'Step 1, Step 1: Identify the number of kids Julia played tag with on Monday and Tuesday.Julia played tag with 18 kids on Monday, and 10 kids on Tuesday.', 'Step 1, Step 1: Find out how many kids Julia played tag with on Monday.Julia played tag with 18 kids on Monday.', 'Step 1, Step 1: Find out how many kids Julia played tag with on Monday.Julia played tag with 18 kids on Monday.', 'Step 1, Step 1: Find out how many kids Julia played with on Monday. She played with 18 kids.', 'Step 1, Step 1: Find out how many kids Julia played tag with on Monday.Number of kids Julia played tag with on Monday = Total number of kids playing tag on Monday = 18.']
correct
likely
inco

In [ ]:
# For bulk solving questions
import json
import numpy as np
import time
## load the test set
data_path = "/content/drive/Shareddrives/517 nlp project/data/SVAMP/test.json"
# load the data
data = json.load(open(data_path))
# randomly sample 60
np.random.seed(0)
data = np.random.choice(data, 60, replace=False)
def eval_sample(question, answer):
    start_time = time.time()
    ys, info = solve(question, 6, 3, 3)
    gen_answer = extract_answer(ys[0])
    if np.abs(gen_answer - answer) < 1e-4:
        correct = 1
    else:
        correct = 0
    end_time = time.time()
    return {
        'question': question,
        'answer': answer,
        'gen_answer': gen_answer,
        'correctness': correct,
        'time': end_time - start_time,
        'info': info
    }

from tqdm import tqdm
results = []
for sample in tqdm(data):
    results.append(eval_sample(sample['question'], sample['answer']))

# save results to /content/drive/Shareddrives/517 nlp project/data/SVAMP/tot_result.json
with open('/content/drive/Shareddrives/517 nlp project/data/SVAMP/tot_result.json', 'w') as f:
    json.dump(results, f)

# get the accuracy
correct = 0
for result in results:
    if result['correctness'] == 1:
        correct += 1
print(correct / len(results))

# get the latency
latencies = []
for result in results:
    latencies.append(result['time'])
print(np.mean(latencies))


 80%|████████  | 48/60 [1:58:20<22:59, 114.97s/it]

In [ ]:
results[4]

NameError: name 'results' is not defined